In [ ]:
# importing libraries
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
column_rename_dict = {
    "PM 2,5": "PM2.5",
    "Partículas Respiráveis (<2,5µm)(µg/m3)": "PM2.5",
    "Partículas Inaláveis (<10µm)(µg/m3)": "PM10",
    "Temperatura (ºC)": "Temperatura",
    "Tº": "Temperatura",
    "Radiação Solar (W/m2)": "Radiação Solar",
    "Monóxido de Carbono (ppm)": "Monóxido de Carbono",
    "CO": "Monóxido de Carbono",
    "Velocidade Escalar do Vento (m/s)": "Velocidade do Vento",
    "Direção Escalar do Vento (grau)": "Direção do Vento",
    "Umidade relativa (%)": "Umidade Relativa",
    "Precipitação Pluviométrica (mm)": "Precipitação",
    "Pressão Atmosférica (mmHg)": "Pressão Atmosférica",
    "Dióxido de enxofre (ppb)": "Dióxido de Enxofre",
    "Ozônio (ppb)": "Ozônio",
    "Monóxido de Nitrogênio (ppb)": "Monóxido de Nitrogênio",
    "Óxidos de Nitrogênio (ppb)": "Óxidos de Nitrogênio",
    "Dióxido de Nitrogênio (ppb)": "Dióxido de Nitrogênio",
    "HCT (ppm)": "HCT",
    "CH4 (ppm)": "Metano",
    "HCNM (ppm)": "HCNM",
    "Monóxido de Carbono Média Móvel (ppm)": "Monóxido de Carbono Média Móvel",
    "Data": "Data",
    "Hora": "Hora",
    "Média Horária": "Média Horária",
    "Seno da direção do vento": "Seno da Direção do Vento",
    "Cosseno da direção do vento": "Cosseno da Direção do Vento",
    "Seno ponderado": "Seno Ponderado",
    "Cosseno ponderado": "Cosseno Ponderado",
    "PM10": "PM10",
    "NO2": "Dióxido de Nitrogênio",
    "O3": "Ozônio",
    "SO2": "Dióxido de Enxofre",
    "DV": "Direção do Vento",
    "PP": "Pressão Atmosférica",
    "PA": "Precipitação",
    "VV": "Velocidade do Vento",
    "RS": "Radiação Solar",
    "UR": "Umidade Relativa"
}

In [ ]:
import os
import pandas as pd

main_folder = "data"
station_folders = [f for f in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, f))]

def get_skiprows(year, station):
    if year == 2021 and station == "Cascata":
        return [0, 1, 3]
    elif year >= 2015:
        return [0, 2]
    else:
        return None

for station_folder in reversed(station_folders):
    base_path = os.path.join(main_folder, station_folder)
    files = [f for f in os.listdir(base_path) if f.endswith(".xlsx")]
    
    for file in reversed(files):
        year = int(file.split("_")[1].split(".")[0])
        file_path = os.path.join(base_path, file)
        skiprows = get_skiprows(year, station_folder)
        df = pd.read_excel(file_path, skiprows=skiprows)
            
        if year < 2015:
            df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y', errors='coerce')
            df['Hora'] = pd.to_datetime(df['Hora'], format='%H:%M:%S', errors='coerce').dt.time
            
            # Combinar 'Data' e 'Hora' em 'Data e Hora'
            df['Data e Hora'] = pd.to_datetime(df['Data'].astype(str) + ' ' + df['Hora'].astype(str), errors='coerce')
            df = df.drop(columns=['Data', 'Hora'])
        else:
            df.rename(columns={df.columns[0]: 'Data e Hora'}, inplace=True)
            df['Data e Hora'] = pd.to_datetime(df['Data e Hora'], format='%d-%m-%Y %H:%M:%S', errors='coerce')
        
        df = df[df['Data e Hora'].notna()]
        df.set_index('Data e Hora', inplace=True)
        
        valid_columns = df.columns.intersection(column_rename_dict.keys())
        df.rename(columns={col: column_rename_dict[col] for col in valid_columns}, inplace=True)
        
        df = df.dropna(axis=1, how='all')

        # Verificar a quantidade de dados ausentes de PM2.5
        pm25_missing = df['PM2.5'].isna().sum()
        pm25_total = len(df)
        print(f"Estação: {station_folder}, Ano: {year}, Dados ausentes de PM2.5: {pm25_missing}/{pm25_total}")
        
        # Salvar o arquivo tratado em uma pasta separada
        treated_folder = os.path.join("tratados", station_folder)
        os.makedirs(treated_folder, exist_ok=True)
        treated_file_path_csv = os.path.join(treated_folder,"csv", f"{station_folder}_{year}_treated.csv")
        treated_file_path_xlsx = os.path.join(treated_folder,"escel", f"{station_folder}_{year}_treated.xlsx")
        df.to_csv(treated_file_path_csv)
        df.to_excel(treated_file_path_xlsx)

    # Combinar os dados tratados de todos os anos para cada estação
    treated_files = [f for f in os.listdir(treated_folder) if f.endswith(".csv")]
    treated_dfs = []
    for treated_file in treated_files:
        treated_file_path = os.path.join(treated_folder, treated_file)
        treated_df = pd.read_csv(treated_file_path, index_col='Data e Hora', parse_dates=True)
        treated_dfs.append(treated_df)

    combined_treated_data = pd.concat(treated_dfs, ignore_index=False, axis=0)
    combined_treated_data.reset_index(inplace=True)
    combined_treated_data.rename(columns={'index': 'Data e Hora'}, inplace=True)
    
    combined_treated_data.to_csv(f"{station_folder}_combined_treated.csv", index=False)
    combined_treated_data.to_excel(f"{station_folder}_combined_treated.xlsx", index=False)


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

treated_folder = "tratados"
station_folders = [f for f in os.listdir(treated_folder) if os.path.isdir(os.path.join(treated_folder, f))]

for station_folder in station_folders:
    print(f"Análise descritiva para a estação: {station_folder}")
    
    treated_files = [f for f in os.listdir(os.path.join(treated_folder, station_folder)) if f.endswith(".csv")]
    
    for treated_file in treated_files:
        year = int(treated_file.split("_")[1])
        treated_file_path = os.path.join(treated_folder, station_folder, treated_file)
        df = pd.read_csv(treated_file_path, parse_dates=['Data e Hora'])
        
        if 'PM2.5' in df.columns:
            print(f"\nAno: {year}")
            
            # Estatísticas descritivas básicas
            pm25_stats = df['PM2.5'].describe()
            print("Estatísticas descritivas para PM2.5:")
            print(pm25_stats)
            
            # Verificar a distribuição dos valores de PM2.5 usando histograma
            plt.figure(figsize=(8, 6))
            plt.hist(df['PM2.5'], bins=20, edgecolor='black')
            plt.xlabel('Valor de PM2.5')
            plt.ylabel('Frequência')
            plt.title(f'Distribuição dos valores de PM2.5 - Estação {station_folder} - Ano {year}')
            plt.tight_layout()
            plt.show()
            
            # Verificar a distribuição dos valores de PM2.5 usando gráfico de densidade
            plt.figure(figsize=(8, 6))
            df['PM2.5'].plot(kind='kde')
            plt.xlabel('Valor de PM2.5')
            plt.ylabel('Densidade')
            plt.title(f'Distribuição dos valores de PM2.5 - Estação {station_folder} - Ano {year}')
            plt.tight_layout()
            plt.show()
            
            # Identificar possíveis outliers
            q1 = df['PM2.5'].quantile(0.25)
            q3 = df['PM2.5'].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - (1.5 * iqr)
            upper_bound = q3 + (1.5 * iqr)
            outliers = df[(df['PM2.5'] < lower_bound) | (df['PM2.5'] > upper_bound)]
            print(f"\nPossíveis outliers para PM2.5 ({len(outliers)} ocorrências):")
            print(outliers['PM2.5'])
        
        else:
            print(f"\nAno: {year}")
            print("A coluna 'PM2.5' não está presente nos dados.")
    
    print("\n" + "-" * 50)


In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

main_folder = "data"
station_folders = [f for f in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, f))]

for station_folder in reversed(station_folders):
    combined_data = pd.read_excel(f"{station_folder}_combined.xlsx")
    # Especificando o formato correto da data
    combined_data['Data e Hora'] = pd.to_datetime(combined_data['Data e Hora'], format='%Y-%m-%d %H:%M:%S', errors='coerce').dt.date
    combined_data = combined_data.set_index('Data e Hora')
    
    # Aumentando o tamanho do gráfico
    plt.figure(figsize=(10, 6))  
    
    combined_data['PM2.5'].plot()
    plt.title(f"{station_folder} PM2.5")
    plt.show()
